In [19]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import expon, uniform, norm

# Part a

In [20]:
HOURS_OPEN = 8
SECONDS_OPEN = HOURS_OPEN * 3600
ARRIVALS_PER_MINUTE = 20
MEAN_SERVICE_TIME = 3 * 60

LAMBDA = ARRIVALS_PER_MINUTE / 60
MU = 1 / MEAN_SERVICE_TIME

MAX_WAIT_TIME = 5
MAX_ABANDONMENT_RATE = 0.02

MIN_PATIENCE_TIME = 0
MAX_PATIENCE_TIME = 6 * 60

In [27]:
arrival_times = np.cumsum(np.random.exponential(scale=1/LAMBDA, size=int(ARRIVALS_PER_MINUTE * 60 * HOURS_OPEN)))

# Assuming calls are answered instantly, call end times are just the arrival times plus service time
call_end_times = arrival_times + MEAN_SERVICE_TIME

# Time points to evaluate the number of calls in progress
time_points = np.arange(0, SECONDS_OPEN + 1, 1)
num_calls_in_progress_a = np.zeros_like(time_points)

# Count the number of calls in progress at each time point
for i, time_point in enumerate(time_points):
    num_calls_in_progress_a[i] = np.sum((arrival_times <= time_point) & (call_end_times > time_point))

In [28]:
call_data = pd.DataFrame({
    'Time (seconds)': time_points,
    'Number of Calls in Progress': num_calls_in_progress_a
})
call_data

,Time (seconds),Number of Calls in Progress
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
28796,28796,25
28797,28797,24
28798,28798,23
28799,28799,23


In [29]:
# Plotting
fig = go.Figure(data=[
    go.Scatter(x=call_data['Time (seconds)'], y=call_data['Number of Calls in Progress'], mode='lines', name='Calls in Progress')
])

# Update the layout
fig.update_layout(
    title="Number of Calls in Progress Over Time",
    xaxis_title="Time (seconds)",
    yaxis_title="Number of Calls in Progress",
    xaxis=dict(range=[0, SECONDS_OPEN]),
    yaxis=dict(range=[0, num_calls_in_progress_a.max() + 1])
)

# Show the figure
fig.show()

In [30]:
from scipy.stats import norm

def plot_normal_distribution_fit(num_calls_in_progress):
    # Fit a normal distribution to the num_calls_in_progress data
    mu, std = norm.fit(num_calls_in_progress)
    x_values = np.linspace(min(num_calls_in_progress), max(num_calls_in_progress), 100)
    pdf = norm.pdf(x_values, mu, std)

    # Create the figure and add the histogram for call distribution
    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=num_calls_in_progress,
        histnorm='probability density',
        name='Call Distribution',
        nbinsx=60,  # Adjust the number of bins as necessary
        marker_color='lightblue',
        opacity=0.75
    ))

    # Add the scatter plot for the normal distribution fit
    fig.add_trace(go.Scatter(
        x=x_values,
        y=pdf,
        name='Normal Distribution Fit',
        line=dict(color='red', width=2)
    ))

    # Update the layout of the figure
    fig.update_layout(
        title='Normal Distribution Fit to Number of People on Call',
        xaxis_title='Number of People on Call',
        yaxis_title='Probability Density',
        bargap=0.2  # Adjust the gap between bars if necessary
    )

    # Show the figure
    fig.show()

# Assuming that num_calls_in_progress is already defined from the previous code
plot_normal_distribution_fit(num_calls_in_progress_a)

In [31]:
average_calls_in_progress = num_calls_in_progress_a.mean()
print(average_calls_in_progress)

peak_max = num_calls_in_progress_a.max()
print(peak_max)

percentiles = np.percentile(num_calls_in_progress_a, [25, 50, 75, 90, 95])
print(percentiles)

variance_calls_in_progress_a = np.var(num_calls_in_progress_a)
variance_calls_in_progress_a

59.975278636158464
82
[54. 60. 66. 71. 74.]


70.02154086278061

## Simulation of the Call Center 

In [33]:
def simulate_call_center(num_agents, num_calls, seed=42):
    np.random.seed(seed)
    # Generate call arrivals
    arrival_times = np.cumsum(np.random.exponential(scale=1/LAMBDA, size=num_calls))
    # Service times and patience times
    service_times = np.random.exponential(scale=MEAN_SERVICE_TIME, size=num_calls)
    patience_times = uniform(loc=MIN_PATIENCE_TIME, scale=MAX_PATIENCE_TIME).rvs(size=num_calls)

    # Initialize agents to be ready at time 0
    agents = np.zeros(num_agents)
    wait_times = []
    abandonments = 0

    for i in range(num_calls):
        if arrival_times[i] > SECONDS_OPEN:  # If the call arrives after closing, do not process
            break
        # Check for the first available agent
        available_agent = np.where(agents <= arrival_times[i])[0]
        if available_agent.size > 0:
            agent_index = available_agent[0]
            agents[agent_index] = arrival_times[i] + service_times[i]  # Update agent's next available time
            wait_times.append(0)  # No wait time since the agent is immediately available
        else:
            # All agents are busy, check if any call will be abandoned
            next_available_time = np.min(agents)
            wait_time = next_available_time - arrival_times[i]
            if wait_time > patience_times[i]:
                abandonments += 1
            else:
                # Wait for the next agent to be available
                agent_index = np.argmin(agents)
                agents[agent_index] += service_times[i]  # Update agent's next available time
                wait_times.append(wait_time)

    average_wait_time = np.mean(wait_times) if wait_times else 0
    abandonment_rate = abandonments / num_calls

    return {
        'num_agents': num_agents,
        'average_wait_time': average_wait_time,
        'abandonment_rate': abandonment_rate,
        'abandonments': abandonments,
        'total_calls': num_calls,
        'wait_times': wait_times
    }

In [34]:

# Let's simulate call centers for a range of agents and collect the data
results = []
for num_agents in range(60, 76):  # Simulating for 60 to 67 agents
    simulation_data = simulate_call_center(num_agents, ARRIVALS_PER_MINUTE * 60 * HOURS_OPEN)
    results.append(simulation_data)

# Creating a DataFrame from the results
df_parta = pd.DataFrame(results)

# Displaying the DataFrame with only the necessary columns
df_parta  

,num_agents,average_wait_time,abandonment_rate,abandonments,total_calls,wait_times
0,60,17.639843,0.052604,505,9600,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,61,14.488439,0.044375,426,9600,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,62,11.872634,0.036771,353,9600,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,63,10.051799,0.030312,291,9600,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,64,8.056772,0.025104,241,9600,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,65,6.744034,0.019896,191,9600,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,66,5.237669,0.016354,157,9600,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,67,4.159098,0.013125,126,9600,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,68,3.168204,0.010104,97,9600,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,69,2.384960,0.007604,73,9600,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## Analyse Part a based on 95% confidence interval 

In [38]:
# Extracting wait times from the test dataframe
wait_times_a = df_parta[df_parta['num_agents']==60]['wait_times'].iloc[0]  # Assuming the first (and only) row contains the relevant data

# Creating the plot with the extracted wait times
fig_a = px.line(
    x=range(len(wait_times_a)), 
    y=wait_times_a, 
    title='Wait Times Plot', 
    labels={'x': 'Call Index', 'y': 'Wait Time'}
    )

fig_a.show()